In [1]:
import tensorflow as tf

#### 1. 创建队列，并操作里面的元素。
* [run](https://www.tensorflow.org/api_docs/python/tf/Session#run)
```shell
run(
    fetches,
    feed_dict=None,
    options=None,
    run_metadata=None
)
```
* Runs operations and evaluates tensors in fetches.
* This method runs one "step" of TensorFlow computation.
* 

In [7]:
#  创建一个先进先出队列，两个元素，类型为整数。
q = tf.FIFOQueue(2, "int32")
# 初始化队列，需明确调用(init.run())
init = q.enqueue_many(([0, 10],))
# 出队列给x
x = q.dequeue()
y = x + 1
# 加1后再入队列。
q_inc = q.enqueue([y])
with tf.Session() as sess:
    init.run() # 需明确调用
    for _ in range(5):
        # run `q_inc` 将执行‘出队列’，‘+1’，‘入队列’全过程。 
        v, _ = sess.run([x, q_inc]) # 'fetches' can be a list.
        print "q[0]: ", v # v对应x.

q[0]:  0
q[0]:  10
q[0]:  1
q[0]:  11
q[0]:  2


In [8]:
import numpy as np
import threading
import time

#### 2. 这个程序每隔1秒判断是否需要停止并打印自己的ID。

In [9]:
def MyLoop(coord, worker_id):
    while not coord.should_stop(): # 判断当前线程是否需要停止
        if np.random.rand()<0.1:  #　random stop
            print "Stoping from id: %d\n" % worker_id,
            coord.request_stop() # stop all threads. 通知其他线程停止。
        else:
            print "Working on id: %d\n" % worker_id, 
        time.sleep(1)

#### 3. 创建、启动并退出线程。
* tf.train.Coordinator用于协同多个线程一起停止.

In [11]:
coord = tf.train.Coordinator()
# 启动５个线程。
threads = [threading.Thread(target=MyLoop, args=(coord, i, )) for i in xrange(5)]
for t in threads:t.start()
coord.join(threads)　# 等待所有线程退出。

Working on id: 0
Working on id: 1
Working on id: 2
Working on id: 3
Working on id: 4
Working on id: 0
Working on id: 1
Working on id: 2
Working on id: 3
Working on id: 4
Working on id: 0
Working on id: 1
Stoping from id: 2
